In [1]:
from pathlib import Path
import sys
from collections import OrderedDict
import json
import subprocess as sp

import pandas as pd
import numpy as np
import payulator as pl
import voluptuous as vt


%load_ext autoreload
%autoreload 2

# Configure

In [2]:
ROOT = Path("../")
LOAN_DIRS = [
    ROOT / "loans" / "active",
    ROOT / "loans" / "finished",
]
%ls {LOAN_DIR}

ls: cannot access '{LOAN_DIR}': No such file or directory


# Define useful functions

In [3]:
def collect_payment_schedules(loan_dirs):
    """
    Given a list of directory paths (list of strings or Path objects),
    search every subdirectory of every given directory for a file
    named ``"payment_schedule.csv"``.
    Read each such payment schedule as a DataFrame and return
    a dictionary of the form loan_directory_name -> payment schedule
    DataFrame.
    """
    d = {}
    for root in loan_dirs:
        for p in root.iterdir():
            if not p.is_dir():
                continue
#             for q in p.glob("payment_schedule.csv"):
#                 d[q.parent.name] = (
#                     pd.read_csv(q, parse_dates=["payment_date"])
#                 )
            for q in p.glob("parameters.json"):
                try:
                    loan = pl.build_loan(q)
                except vt.MultipleInvalid as e:
                    print(f"Skipping {p} because of an error in its JSON paremeters.")
                    print(e)
                    
                d[loan.code] = loan.summarize()["payment_schedule"]
            
    return d

def compute_balances(loan_dirs, date):
    """
    Read all the payment schedules in all the subdirectories 
    of the given loan directories, and compile their outstanding loan
    balances (pricipals) up to the given date.
    Return the results as a DataFrame with the columns:
    
    - ``"loan_code"``
    - ``"first_payment_date"``: first scheduled payment date of the loan
    - ``"last_payment_date"``: last scheduled payment date of the loan
    - ``"balance_date"``: ``date``
    - ``"balance"``: balance (principal) of the loan up to ``date``
    - ``"total_balance"``: sum of all the balances in the DataFrame
    
    """
    d = collect_payment_schedules(loan_dirs)
    if not d:
        return pd.DataFrame()
    
    rows = []
    for name, f in d.items():
        f = f.set_index("payment_date")
        start_date = f.index[0]
        end_date = f.index[-1]
        s = f.loc[:date, "ending_balance"]
        if not s.empty:
            b = s.iat[-1]
        else:
            b = np.nan
        rows.append([name, start_date, end_date, date, b])
    
    return  (
        pd.DataFrame(rows, columns=["loan_code", "first_payment_date", 
          "last_payment_date", "balance_date", "balance"])
        .assign(total_balance = lambda x: x["balance"].sum())
        .sort_values("first_payment_date")
        .reset_index(drop=True)
    )

def compute_revenue(loan_dirs, start_date=None, end_date=None, freq="M"):
    """
    Use the function :func:`collect_payment_schedules` to collect all
    payment schedule in the given directories.
    Assume each schedule has the format expected by the function
    :func:`payulator.aggregate_payment_schedules`, and apply that
    function at the given frequency to the schedules.
    Slice to the result given start and end dates (inclusive), 
    append a few summary columns, and return the resulting DataFrame
    with the columns
    
    - ``"payment_date"``    
    - ``"principal_payment"``   
    - ``"interest_payment"`` 
    - ``"fee_payment"``
    - ``"total_payment"``
    - ``"revenue"``: interest_payment + fee_payment
    - ``"principal_payment_cumsum"``: cumulative sum of principal_payment  
    - ``"interest_payment_cumsum"``: cumulative sum of interest_payment    
    - ``"total_payment_cumsum"``: cumulative sum of total_payment
    - ``"revenue_cumsum"``: cumulative sum of revenue
    
    """
    # Collect payment schedules from all loans on record
    d = collect_payment_schedules(loan_dirs)
    if not d:
        return pd.DataFrame()
    
    # Aggregate
    return (
        pl.aggregate_payment_schedules(
            d.values(), 
            start_date=start_date, 
            end_date=end_date, 
            freq=freq
        )
        .assign(revenue=lambda x: x.interest_payment + x.fee_payment)
        .assign(revenue_cumsum=lambda x: x.revenue.cumsum())
    )

def estimate_future_fund(loan_dirs, current_date, future_date, current_fund, 
  funding, funding_freq):
    """
    """
    # Build date range from current date to future date using given funding freq.
    # Note that pandas.date_range does not do this exactly.
    date = pd.to_datetime(current_date)
    rng = []
    while date <= pd.to_datetime(future_date):
        rng.append(date)
        date += pd.tseries.frequencies.to_offset(funding_freq)
    
    loans = compute_revenue(root_dir, current_date, future_date, "D").set_index("payment_date")
    return (
        pd.DataFrame(index=rng)
        .assign(start_fund = current_fund)
        .assign(funding = pd.Series(funding, index=rng).shift().fillna(0))
        .assign(funding_cumsum = lambda x: x["funding"].cumsum())
        .assign(loan_revenue_cumsum = lambda x: loans.loc[x.index, "total_payment_cumsum"])
        .assign(fund = lambda x: x["start_fund"] + x["funding_cum"] + x["loan_revenue_cumsum"])
        .drop(["funding"], axis=1)
        .rename_axis("date")
        .reset_index()
    )


# Compute revenue, costs, etc.

In [4]:
compute_revenue(LOAN_DIRS, "2021-11-01")


Skipping ../loans/finished/Swinson-20170831 because of an error in its JSON paremeters.
Kind must be one on ['amortized', 'interest_only'] for dictionary value @ data['kind']


,payment_date,principal_payment,interest_payment,fee_payment,total_payment,principal_payment_cumsum,interest_payment_cumsum,fee_payment_cumsum,total_payment_cumsum,revenue,revenue_cumsum
0,2021-11-30,2186.89,1629.53,0,3816.42,2186.89,1629.53,0,3816.42,1629.53,1629.53
1,2021-12-31,2205.01,1611.42,0,3816.43,4391.90,3240.95,0,7632.85,1611.42,3240.95
2,2022-01-31,2223.27,1593.16,0,3816.43,6615.17,4834.11,0,11449.28,1593.16,4834.11
3,2022-02-28,2241.68,1574.75,0,3816.43,8856.85,6408.86,0,15265.71,1574.75,6408.86
4,2022-03-31,32260.24,1556.17,0,33816.41,41117.09,7965.03,0,49082.12,1556.17,7965.03
5,2022-04-30,2278.97,1287.46,0,3566.43,43396.06,9252.49,0,52648.55,1287.46,9252.49
6,2022-05-31,62297.85,1268.59,0,63566.44,105693.91,10521.08,0,116214.99,1268.59,10521.08
7,2022-06-30,52316.87,749.55,0,53066.42,158010.78,11270.63,0,169281.41,749.55,11270.63
8,2022-07-31,2336.06,313.70,0,2649.76,160346.84,11584.33,0,171931.17,313.70,11584.33
9,2022-08-31,2355.42,294.35,0,2649.77,162702.26,11878.68,0,174580.94,294.35,11878.68


In [5]:
# Summarize
d = {}

start_date = pd.to_datetime("2020-04-01")
end_date = pd.to_datetime("2021-03-31")
TAX_DIR = ROOT / "taxes" / f"{start_date:%Y%m%d}--{end_date:%Y%m%d}"
%ls {TAX_DIR}

d["revenue"] = (
    compute_revenue(LOAN_DIRS, start_date=f"{start_date:%Y-%m-%d}", end_date=f"{end_date:%Y-%m-%d}")
    .revenue
    .sum()
)

# Office expeneses can claim for home office, which occupies 24% of home.
# Download flat account transactions CSV from bank for this.
rent =  (
    pd.read_csv(TAX_DIR / "expenses_office.csv")
    .fillna("nada")
    .loc[lambda x: x["Memo/Description"].str.contains("from A Raichev", case=False)]
    .loc[lambda x: x["TP part"].str.contains("rent", case="False")]
    .loc[: , "Amount"]
    .sum()
    - 25*52  # Rent & utils - utils
)
display(rent)
d["expenses_office"] = rent * (11.34 / 26.025) + 43 * (11.34)

# Next year use the IRD square meter rate for expenses; see 
# https://www.ird.govt.nz/income-tax/income-tax-for-businesses-and-organisations/types-of-business-expenses/using-your-home-for-your-business
# (yearly rent) * (office area i pay for)/(total area i pay for) + 43 * (office area i pay for)

d["expenses_other"] = pd.read_csv(TAX_DIR / "expenses_other.csv").amount.sum() 
d["expenses"] = d["expenses_other"] + d["expenses_office"]
d["gross_profit"] = d["revenue"] - d["expenses"] 
f = (
    pd.DataFrame(d, index=[0])
    .round(0)
    .assign(
        tax=lambda x: 0.28 * x.gross_profit,
        net_profit=lambda x: x.gross_profit - x.tax,
        net_profit_alt=lambda x: x.revenue - x.expenses_other - x.tax,
        net_profit_ratio=lambda x: x.net_profit / x.revenue,
        net_profit_ratio_alt=lambda x: x.net_profit_alt / x.revenue,
    )
)
f.T

expense_receipts/    ir10.pdf        mailed_tax_forms_to_ird_20210518
expenses_office.csv  ir4.pdf         transactions.csv
expenses_other.csv   ir4_signed.pdf
Skipping ../loans/finished/Swinson-20170831 because of an error in its JSON paremeters.
Kind must be one on ['amortized', 'interest_only'] for dictionary value @ data['kind']


13375.0

,0
revenue,14407.000000
expenses_office,6316.000000
expenses_other,3077.000000
expenses,9393.000000
gross_profit,5015.000000
tax,1404.200000
net_profit,3610.800000
net_profit_alt,9925.800000
net_profit_ratio,0.250628
net_profit_ratio_alt,0.688957


# Compute assets

In [17]:
# Loan balances
d = {}

date = "2021-03-31"
loan_balances = compute_balances(LOAN_DIRS, date)
display(loan_balances)

d["outstanding_balances"] = loan_balances.balance.sum()
d["cash"] = 18_685

# Compute other assets excluding depreciation

assets = 0 
tax_dir = ROOT / "taxes"
for subdir in sorted(tax_dir.iterdir()):
    for path in subdir.iterdir():
        if path.name == "expenses_other.csv":
            asset = (
                pd.read_csv(path)
                .loc[lambda x: x.is_asset.str.contains("yes", case=False)]
                .amount
                .sum()
            )
            assets += asset

d["non_loan_assets"] = assets

f = (
    pd.DataFrame(d, index=[0])
    .round(0)
    .assign(total_assets=lambda x: x.sum(axis=1))
)
f.T

Skipping ../loans/finished/Swinson-20170831 because of an error in its JSON paremeters.
Kind must be one on ['amortized', 'interest_only'] for dictionary value @ data['kind']


,loan_code,first_payment_date,last_payment_date,balance_date,balance,total_balance
0,EBH-20170927,2017-11-06,2018-10-06,2021-03-31,0.00,130632.17
1,Resilio-20180123,2018-02-06,2021-01-06,2021-03-31,0.00,130632.17
2,Vallyon-20180102,2018-02-06,2020-01-06,2021-03-31,0.00,130632.17
3,Haldane-Willis-20180203,2018-03-06,2021-02-06,2021-03-31,0.00,130632.17
4,C-Vallyon-20180308,2018-04-06,2019-04-06,2021-03-31,0.00,130632.17
5,EBH-20180224,2018-04-06,2019-03-06,2021-03-31,0.00,130632.17
6,C-Vallyon-20180722,2018-09-06,2019-04-06,2021-03-31,0.00,130632.17
7,EBT-20180916,2018-11-06,2019-03-06,2021-03-31,0.00,130632.17
8,K-Riegle-van-West-20190316,2019-04-06,2020-03-06,2021-03-31,0.00,130632.17
9,EBT-20190303,2019-04-06,2020-03-06,2021-03-31,0.00,130632.17


,0
outstanding_balances,130632.0
cash,18685.0
non_loan_assets,584.0
total_assets,149901.0
